# HARK and SSJ example

By William Du (wdu9@jhu.edu) 


- This notebook demonstrates how HARK and the Sequence Space Jacobian toolkit can be combined to solve HANK models.
    - And is heavily drawn from this [SSJ notebook](https://github.com/shade-econ/nber-workshop-2022/blob/main/Tutorials/Tutorial%201%20Fiscal%20Policy.ipynb) .

- HARK was designed to solve rich microeconomic models, including:
    - Lifecycle models
    - Discrete-continuous choice
    - Portfolio choice

</br>

- SSJ was created to solve HANK models with incredibly speed and ease

- Connecting HARK to SSJ allows us to solve macro models with richer micro features.


# Model


To begin, a basic HANK model is introduced. The model follows the [lecture slides](https://github.com/shade-econ/nber-workshop-2022/blob/main/Lectures/Lecture%204%20Fiscal%20Policy.pdf) from the authors' heterogeneous agent workshop.

## Basic model features

- Heterogenous agent households that consume/save into one asset

- Households face idiosyncratic income shocks

- Cobb-Douglas production function with labor and capital

- Sticky wages

- To pay its expenditures, government taxes labor income and rolls over its debt


# Bringing Model to Code

To implement the model, we first need to import some basic, commonly used numeric tools (e.g. numpy and scipy), as well as an AgentType subclass from HARK. Additionally, we import the SSJ toolkit to apply as an additional layer on top of the microeconomic model.

In [ ]:
# Basic tools
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy import optimize

# HARK agents
from HARK.ConsumptionSaving.ConsNewKeynesianModel import NewKeynesianConsumerType

# Sequence space Jacobian toolkit
from sequence_jacobian.classes import JacobianDict, SteadyStateDict
import sequence_jacobian as sj  # SSJ will allow us to define blocks, models, compute IRFs, etc

### Calibrate Steady State Values

Next, we calibrate the steady state values of some outcomes, and calculate a few values implied by them.

In [ ]:
# Steady state calibration
r_ss = 0.03  # Interest rate
G_ss = 0.2  # Government spending
B_ss = 0.2  # Government debt (bonds)
Y_ss = 1.0  # Output

# Implied steady state values
T_ss = G_ss + r_ss * B_ss  # Lump sum taxes to keep debt constant
Z_ss = Y_ss - T_ss  # After tax labor income
F_ss = G_ss - T_ss  # Government budget deficit

### Microeconomic Agent Parameters

To make our microeconomic agents in HARK, we need a dictionary of parameters, defined below. All of these parameters have default values if they were not specified, but we list them here for completeness.

In [ ]:
HANK_dict = {
    "cycles": 0,  # Infinite horizon problem
    # Parameters shared with the perfect foresight model
    "Rfree": 1.0 + r_ss,  # Interest factor on assets
    "LivPrb": [0.99375],  # Survival probability
    # Parameters that specify the income distribution over the lifecycle
    "PermShkStd": [0.06],  # Standard deviation of log permanent shocks to income
    "PermShkCount": 5,  # Number of points in discrete approximation to permanent income shocks
    "TranShkStd": [0.2],  # Standard deviation of log transitory shocks to income
    "TranShkCount": 5,  # Number of points in discrete approximation to transitory income shocks
    # HANK parameters
    "tax_rate": [0.0],  # Assume that labor here is actually after tax income
    "labor": [Z_ss],  # Will be solving the micro model in steady state
    "wage": [1.0],  #
    "UnempPrb": 0.0,  # Probability of unemployment while working
    "IncUnemp": 0.0,  # Unemployment benefits replacement rate
    # Parameters for constructing the "assets above minimum" grid
    "aXtraMax": 50,  # Maximum end-of-period "assets above minimum" value
    "aXtraCount": 100,  # Number of points in the base grid of "assets above minimum"
    "BoroCnstArt": 0.0,
    # Transition matrix simulation parameters
    "mCount": 500,
    "mMax": 100,
    "mMin": 1e-5,
    "mFac": 3,
}

## Find Appropriate Discount Factor

Above, we exogenously calibrated the steady state level of government debt $B_{ss}$. These bonds should be *owned* by someone, so we need our agents to want to hold the same level of assets as there are government bonds. To do so, we specify a function that yields steady state asset holdings as a function of agents' discount factor $\beta$, and another function that simply differences it with our target level of assets. The appropriate discount factor can then be found by performing a bounded search for a root of the difference function.

In [ ]:
# Define a function that computes steady state aggregate assets as a function of the discount factor
def calc_SS_assets_from_discfac(beta):
    HANK_dict["DiscFac"] = beta  # set the proposed discount factor
    TempAgent = NewKeynesianConsumerType(
        **HANK_dict, verbose=False
    )  # make an AgentType instance
    A_ss = TempAgent.compute_steady_state()[0]  # compute steady state assets
    return A_ss


# Define a simple function that differences steady state assets from target assets
def calc_asset_gap(beta):
    return calc_SS_assets_from_discfac(beta) - A_target


# Find the discount factor that makes steady state assets equal (calibrated) steady state government bonds
A_target = B_ss
DiscFac_star = optimize.brentq(calc_asset_gap, 0.85, 0.99)

## Make Agents with Calibrated Discount Factor

Now that we know what the discount factor should be, we can make an AgentType instance that will be used for the general equilibrium analysis. To do so, we simply put $\beta^*$ into our parameter dictionary and make a new instance of NewKeynesianConsumerType using it. To verify that we did what we set out to do, we can check that the output goods market clears (output is accounted for by consumption and government spending) and that the asset market clears (retained assets are accounted for by government bonds). Both conditions should be very close to zero.

In [ ]:
# Create a HARK agent
HANK_dict["DiscFac"] = DiscFac_star
Agent_GE = NewKeynesianConsumerType(**HANK_dict, verbose=False)
A_ss, C_ss = Agent_GE.compute_steady_state()

# Verify market clearing conditions
print("Goods clearing condition:", Y_ss - C_ss - G_ss)
print("Asset clearing condition:", A_ss - B_ss)

## Compute Jacobian Matrices

We can now calculate the Jacobian matrices that characterize consumption and asset responses to an exogenous change in the interest factor (Rfree) or labor supply (labor) that will occur in the future. This is accomplished in HARK with the calc_jacobian method, passing as arguments the name of the parameter that will be changed and the maximum number of periods in the future that a (marginal) change could occur.

The outputs of this method are objects that represent the sensitivity of aggregate consumption and asset holdings (respectively) in each period to the *advance knowledge* of an exogenous change in each parameter.

In [ ]:
start = time.time()

dCdR, dAdR = Agent_GE.calc_jacobian(
    "Rfree", 300
)  # Jacobians of consumption and assets w.r.t. interest factor changes up to 300 periods ahead
dCdZ, dAdZ = Agent_GE.calc_jacobian(
    "labor", 300
)  # Jacobians of consumption and assets w.r.t. labor supply changes up to 300 periods ahead

print("Time to compute Jacobians: {:.3f} seconds.".format(time.time() - start))

## Plot Jacobians

To inspect the Jacobian matrix for aggregate consumption, we can plot some of its rows/columns, representing the consumption response to a change in the interest factor or labor income $T$ periods in the future.

In [ ]:
plt.plot(dCdR.T[0])
plt.plot(dCdR.T[30])
plt.plot(dCdR.T[50])
plt.legend(
    [
        r"Change in $R$ right now",
        r"Change in $R$ 30 periods ahead",
        r"Change in $R$ 50 periods ahead",
    ]
)
plt.xlim(-2, 120)
plt.title("Consumption Response to Future Change in Interest Factor")
plt.xlabel("Time")
plt.ylabel("What is the scale here?")
plt.show()

In [ ]:
plt.plot(dCdZ.T[0])
plt.plot(dCdZ.T[20])
plt.plot(dCdZ.T[50])
plt.xlim(-2, 120)
plt.legend(
    [
        r"Change in $Z$ right now",
        r"Change in $Z$ 30 periods ahead",
        r"Change in $Z$ 50 periods ahead",
    ]
)
plt.xlim(-2, 120)
plt.title("Consumption Response to Future Change in Labor Income")
plt.xlabel("Time")
plt.ylabel("What is the scale here?")
plt.show()

# Specify Government Budget Constraint and Market Clearing Conditions

In [ ]:
# Government Budget Constraint
@sj.simple
def fiscal(B, r, G, Y):
    T = (1 + r) * B(-1) + G - B  # total tax burden
    Z = Y - T  # after tax income
    deficit = G - T
    return T, Z, deficit


# Market Clearing Conditions
@sj.simple
def mkt_clearing(A, B, Y, C, G):
    asset_mkt = A - B
    goods_mkt = Y - C - G
    return asset_mkt, goods_mkt

In [ ]:
# Store Jacobians in JacobianDict Object
Jacobian_dict = JacobianDict({"C": {"Z": CJACZ}, "A": {"Z": AJACZ}})

In [ ]:
# Construct SteadyStateDict object
SteadyState_dict = SteadyStateDict(
    {
        "asset_mkt": 0.0,
        "goods_mkt": 0.0,
        "deficit": G_ss - T_ss,
        "B": B_ss,  # Gov. Debt
        "r": r_ss,  # Interest rate
        "G": G_ss,  # Gov. Spending
        "Y": Y_ss,  # Output
        "A": B_ss,  # Assets
        "C": C_ss,  # Consumption
        "Z": Y_ss - T_ss,  # Income (post-tax)
        "T": T_ss,  # Lump Sum Taxes
    }
)

In [ ]:
T = 300  # <-- the length of the IRF
rho_G = 0.8  # peristence of shock
dG = 0.01 * rho_G ** np.arange(T)
shocks = {"G": dG}

unknowns_td = ["Y"]
targets_td = ["asset_mkt"]

# Create model
ha_lin = sj.create_model([Jacobian_dict, fiscal, mkt_clearing], name="HA Model")

# obtain impulse responses
irfs_G_lin = ha_lin.solve_impulse_linear(
    SteadyState_dict, unknowns_td, targets_td, shocks
)

In [ ]:
def show_irfs(
    irfs_list,
    variables,
    labels=[" "],
    ylabel=r"Percentage points (dev. from ss)",
    T_plot=50,
    figsize=(18, 6),
):
    if len(irfs_list) != len(labels):
        labels = [" "] * len(irfs_list)
    n_var = len(variables)
    fig, ax = plt.subplots(1, n_var, figsize=figsize, sharex=True)
    for i in range(n_var):
        # plot all irfs
        for j, irf in enumerate(irfs_list):
            ax[i].plot(irf[variables[i]][:50], label=labels[j])
        ax[i].set_title(variables[i])
        ax[i].set_xlabel(r"$t$")
        if i == 0:
            ax[i].set_ylabel(ylabel)
        ax[i].legend()
    plt.show()

## Government Spending Shock (Balanced Budget)

In [ ]:
show_irfs([irfs_G_lin], ["G", "Y", "T"])

## Government Spending shock (deficit financed)

In [ ]:
rho_B = 0.9
dB = np.cumsum(dG) * rho_B ** np.arange(T)
shocks_B = {"G": dG, "B": dB}

irfs_B_lin = ha_lin.solve_impulse_linear(
    SteadyState_dict, unknowns_td, targets_td, shocks_B
)

In [ ]:
show_irfs(
    [irfs_G_lin, irfs_B_lin],
    ["G", "Y", "T", "deficit", "goods_mkt"],
    labels=["balanced budget", "deficit financed"],
)

## Nonlinear Impulse Responses

In [ ]:
# NOTE: These cells have been commented out because they rely on files that are not present in the HARK repo.

# def hh(Z):
#    C,A = Agent_GE.calc_agg_path(Z,300)
#    return C,A

In [ ]:
# from Misc_Utilities.simple_block import exog # import wrapper so SSJ can identify HARK household object

# hh_block = exog(Jacobian_dict,hh,hh)
# hh_block.name = hh

# ha = sj.create_model([hh_block,fiscal,mkt_clearing], name="HA Model")
# irfs_G_nonlin = ha.solve_impulse_nonlinear(SteadyState_dict, unknowns_td, targets_td, shocks)


In [ ]:
# plt.plot(irfs_G_nonlin['Y'], label ='nonlinear')
# plt.plot(irfs_G_lin['Y'], label = 'linear', linestyle='--')
# plt.xlim(-1,50)
# plt.legend()
# plt.title('Nonlinear Impulse responses of output')
# plt.show()

In [ ]:
# show_irfs([irfs_G_lin,irfs_G_nonlin], ['G', 'Y','T'], ['Linear', 'Nonlinear'])

# Why does HANK matter?

Let us compare HANK vs TANK vs RANK.

In this section we will define household blocks for a representative agent and a two agent. 



## Specify a representative agent household

In [ ]:

@sj.solved(unknowns={'C': 1, 'A': 1},
           targets=["euler", "budget_constraint"])  # solver="broyden_custom")
def hh_ra(C, A, Z, eis, beta, r):
    euler = (beta * (1 + r(+1)))**(-eis) * C(+1) - C
    budget_constraint = (1 + r) * A(-1) + Z - C - A
    MPC = 0
    return euler, budget_constraint, MPC

ra = sj.create_model([hh_ra, fiscal, mkt_clearing], name="Representative agent model")



## Specify a two agent household

In [ ]:
@sj.solved(unknowns={'C_RA': 1, 'A': 1},
           targets=["euler", "budget_constraint"])  # , solver="broyden_custom")
def hh_ta(C_RA, A, Z, eis, beta, r, lam):
    euler = (beta * (1 + r(+1))) ** (-eis) * C_RA(+1) - C_RA      # consumption of infinitely lived household
    C_H2M = Z   # computes consumption of an hand to mouth agent
    C = (1 - lam) * C_RA + lam * C_H2M
    budget_constraint = (1 + r) * A(-1) + Z - C - A
    MPC = 0
    return euler, budget_constraint, C_H2M, C, MPC

ta = sj.create_model([hh_ta, fiscal, mkt_clearing], name="Two agent model")

## Calibrate and find steady states of TANK and RANK models.

In [ ]:
calibration = {'eis': 0.5,  # EIS
               'G': G_ss,  # Government spending
               'B': B_ss,  # Government debt
               'Y': Y_ss,  # Output
              'r': r_ss,}  # Number of productivity grid points


In [ ]:
calibration_ra = calibration.copy()
calibration_ra['beta'] = 1 / (1 + calibration_ra['r'])
calibration_ra['B'] = calibration['B']

unknowns_ra_ss = {'C': 1., 'A': 0.8}
targets_ra_ss = {'budget_constraint': 0., 'asset_mkt': 0.}

ss_ra = ra.solve_steady_state(calibration_ra, unknowns_ra_ss, targets_ra_ss, dissolve=['hh_ra'])

In [ ]:
calibration_ta = calibration_ra.copy()
calibration_ta['lam'] = 0.25
unknowns_ta_ss = {'C_RA': 1., 'A': 0.8}

ss_ta = ta.solve_steady_state(calibration_ta, unknowns_ta_ss, targets_ra_ss, dissolve=['hh_ta'])

## Impulse responses to deficit financed G shocks: HANK vs TANK vs RANK

In [ ]:
irfs_ra = ra.solve_impulse_linear(ss_ra, unknowns_td, targets_td, shocks_B)
irfs_ta = ta.solve_impulse_linear(ss_ta, unknowns_td, targets_td, shocks_B)
show_irfs([irfs_B_lin, irfs_ra, irfs_ta], variables=['deficit', 'Y', 'C'], labels=['HANK', 'RANK', 'TANK'])

# Simulating time series

## Let us simulate the economy when it is subject to deficit financed government spending shocks.

In [ ]:
from estimation.plots import plot_timeseries
from estimation.routines import simulate


outputs = ['Y', 'C', 'deficit','G'] # variables to be simulated

impulses_ha = {}
impulses_ra = {}
impulses_ta = {}

shk_std = .001 # standard deviation of shock
rho_G = .8 # persistence of G shock
rho_B = 0.9 # persistence of Deficit shock 

dG = shk_std * .8 ** np.arange(T) # G Shock series
dB = np.cumsum(dG) * rho_B ** np.arange(T) # deficit shock series

own_shock = {"G": shk_std * rho_G ** np.arange(T) ,"B": dB }
impulses_ha['ha'] = ha_lin.solve_impulse_linear(SteadyState_dict, unknowns_td, targets_td, own_shock)
impulses_ta['ta'] = ta.solve_impulse_linear(ss_ta, unknowns_td, targets_td, own_shock)
impulses_ra['ra'] = ra.solve_impulse_linear(ss_ra, unknowns_td, targets_td, own_shock)


T_sim = 80  # simulate for 80 quarters
data_simul_ha = simulate(list(impulses_ha.values()), outputs, T_sim)
data_simul_ta = simulate(list(impulses_ta.values()), outputs, T_sim)
data_simul_ra = simulate(list(impulses_ra.values()), outputs, T_sim)


# Comparing time series of HANK, TANK, and RANK

In [ ]:

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(13, 5))
axes[0].plot(data_simul_ha['Y'])
axes[0].plot(data_simul_ta['Y'])
axes[0].plot(data_simul_ra['Y'])
axes[0].set_title('Output')
axes[0].set_xlabel('Quarters')
axes[1].plot(data_simul_ha['G'])
axes[1].plot(data_simul_ta['G'])
axes[1].plot(data_simul_ra['G'])
axes[1].set_title('Government Spending')
axes[1].set_xlabel('Quarters')
axes[2].plot(data_simul_ha['deficit'])
axes[2].plot(data_simul_ta['deficit'])
axes[2].plot(data_simul_ra['deficit'])
axes[2].set_title('Deficit')
axes[2].set_xlabel('Quarters')
fig.tight_layout()